In [1]:
# -*- coding: utf-8 -*-
# ## Markdown to JSON for AT‑命令 CSV

import pandas as pd
import json
import re

def parse_markdown_nested(md_text: str):
    """
    将 markdown 列表解析为嵌套字典，支持主键 + 子键结构，
    并保留主键本身的描述在 "__desc__" 字段里（如果有的话）。
    
    假设结构类似：
    - `<key>`：主描述
      - `子key1`: 子描述1
      - `子key2`: 子描述2
    - `<other>`：描述
    
    返回形如：
    {
      "<key>": {
         "__desc__": "主描述",
         "子key1": "子描述1",
         "子key2": "子描述2"
      },
      "<other>": {
         "__desc__": "描述"
      }
    }
    """
    result = {}
    current_key = None

    # 分行
    lines = md_text.splitlines()
    for raw in lines:
        # 跳过空行
        if not raw.strip():
            continue

        indent = len(raw) - len(raw.lstrip())
        content = raw.strip()

        # 匹配 “- `<key>`：描述” 这种键值对格式
        m = re.match(r"[-*]\s*`?([^`：:]+)`?\s*[：:]\s*(.*)", content)
        if m and indent == 0:
            # 主键行
            key, desc = m.groups()
            key = key.strip()
            desc = desc.strip()
            current_key = key
            result[current_key] = {"__desc__": desc}
        elif m and indent > 0 and current_key is not None:
            # 子键行（缩进 + 列表形式）
            subkey, subdesc = m.groups()
            subkey = subkey.strip()
            subdesc = subdesc.strip()
            # 确保主键是 dict 结构
            if not isinstance(result[current_key], dict):
                result[current_key] = {"__desc__": ""}
            result[current_key][subkey] = subdesc
        else:
            # 如果遇到不匹配的行，可以尝试作为主键补充描述或忽略
            # 下面这段是可选的：把非标准行合并到主键描述
            if current_key is not None:
                # 把 content 附加到主键的描述里（如果子项不匹配）
                prev = result[current_key].get("__desc__", "")
                combined = prev + ("\n" if prev else "") + content
                result[current_key]["__desc__"] = combined

    return result

def process_dataframe(df: pd.DataFrame):
    json_col = []
    for idx, row in df.iterrows():
        md = row.get("参数1", "")
        try:
            parsed = parse_markdown_nested(md)
            json_str = json.dumps(parsed, ensure_ascii=False)
        except Exception as e:
            print(f"第 {idx} 行解析失败：{e}")
            json_str = ""
        json_col.append(json_str)
    df["参数json"] = json_col
    return df

def main():
    # 路径按你实际调整
    in_csv = "at_commands.csv"
    out_csv = "at_commands_with_json.csv"

    df = pd.read_csv(in_csv, dtype=str).fillna("")
    df = process_dataframe(df)
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return df

if __name__ == "__main__":
    df = main()
    # 打印几行看看效果
    print(df[["命令", "参数1", "参数json"]].head(10))


        命令                                                参数1  \
0      ATI  - `<manufacturer>`：模组厂商信息、产品名称、版本号\r\n\r\n- `<...   
1  AT+CSMS  - `<service>`：\r\n  - `0`: GSM03.40 and GSM03....   

                                              参数json  
0  {"<manufacturer>": {"__desc__": "模组厂商信息、产品名称、版...  
1  {"<service>": {"__desc__": "", "0": "GSM03.40 ...  
